### spam

In [1]:
import os
import pickle
import numpy as np
from collections import Counter

N = 33716 #5172

def make_Dictionary(root_dir):
    emails_dirs = [os.path.join(root_dir,f) for f in os.listdir(root_dir)]    
    all_words = []       
    for emails_dir in emails_dirs:
        dirs = [os.path.join(emails_dir,f) for f in os.listdir(emails_dir)]
        for d in dirs:
            emails = [os.path.join(d,f) for f in os.listdir(d)]
            for mail in emails:
                with open(mail, encoding = 'ISO-8859-1') as m:
                    for line in m:
                        words = line.split()
                        all_words += words
    dictionary = Counter(all_words)
    list_to_remove = dictionary.keys()
    
    for item in list(dictionary):
        if item.isalpha() == False: 
            del dictionary[item]
        elif len(item) == 1:
            del dictionary[item]
    dictionary = dictionary.most_common(3000)
    
    np.save('dict_enron.npy',dictionary) 
    
    return dictionary

def extract_features_realdata(root_dir):
    emails = [os.path.join(root_dir,f) for f in os.listdir(root_dir)]  
    docID = 0
    features_matrix = np.zeros((len(emails),3000))
    for mail in emails:
        print (mail)
        with open(mail, encoding = 'ISO-8859-1') as m:
            all_words = []
            for line in m:
                words = line.split()
                all_words += words
                for word in all_words:
                    wordID = 0
                    for i,d in enumerate(dictionary):
                        if d[0] == word:
                            wordID = i
                            features_matrix[docID,wordID] = all_words.count(word)
        docID = docID + 1                
    return features_matrix

In [2]:
dictionary = np.load('dictionary.npy')

In [3]:
# load the model from disk
filename = 'spam_model.sav'

loaded_model = pickle.load(open(filename, 'rb'))

In [4]:
real_dir = "test"
real_matrix = extract_features_realdata(real_dir)
print (real_matrix.shape)

test\Sample_000001_00000211.txt
(1, 3000)


In [5]:
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.predict(real_matrix)
print(result)

[ 1.]


### URL

In [6]:
# load the classification model from disk
filename = 'url_malicious_model.sav'

loaded_model = pickle.load(open(filename, 'rb'))

In [7]:
# load the tfidf model from disk
filename1 = 'url_tfidf_model.sav'

loaded_model1 = pickle.load(open(filename1, 'rb'))

In [8]:
import time

t0 = time.time()

X_real = ["wikipedia.com",
          "google.com/search=faizanahmad",
          "pakistanifacebookforever.com/getpassword.php/",
          "www.radsport-voggel.de/wp-admin/includes/log.exe",
          "ahrenhei.without-transfer.ru/nethost.exe",
          "www.itidea.it/centroesteticosothys/img/_notes/gum.exe",
          "xn--cgopolygon-wy2e.com/login.php"]

X_real = loaded_model1.transform(X_real)
y_real = loaded_model.predict(X_real)

t1 = time.time()
print ("cost time :", t1-t0)
print (y_real)

cost time : 0.03509259223937988
[0 0 0 1 1 1 1]


### fraud

In [10]:
import os
import re
import csv
import glob
import operator
import collections
import numpy as np
from collections import Counter


In [11]:
# split word to ( , )

train_dir = 'keywordmatrix'

name_list = os.listdir(train_dir)
full_list = [os.path.join(train_dir,i) for i in name_list]
time_sorted_list = sorted(full_list, key=os.path.getctime)
print(time_sorted_list[-1])

all_words = []
for email in time_sorted_list:
    with open(email, encoding = 'ISO-8859-1') as m:
        a = []
        for i,line in enumerate(m):
            if i != 1:
                words_ = line
                pattern3 = re.compile("[^\w\d]+")
                words_ = pattern3.sub(' ',words_)
                words = words_.split()
                for j in range(len(words)):
                    a.append(words[j])
    all_words.append(a)
print ('size of letter :', len(all_words))

keywordmatrix\0001.1999-12-10.farmer.ham.txt
size of letter : 254


In [12]:
import numpy as np

np.save('keywordmatrix.npy', all_words)

cut_programs = np.load('keywordmatrix.npy')

voc_dict = np.load('keyword_voc_dict.npy')

In [13]:
# word filter

example_doc = []

for letter in cut_programs:
    example_line = ''
    for w in letter:
        if w in voc_dict:
            example_line += w+' '
        
    example_doc.append(example_line)

print( example_doc[0] )

com body font 14px 20px Helvetica Arial sans serif margin padding text align center webkit text size adjust none p padding 10px h1 img max 100 height auto vertical align bottom h2 font size 22px line height 28px margin 12px h3 margin 12px background none padding border none wrapper 600px margin auto 10px auto text align left input button border none button display inline block font weight 500 font size 16px line height 42px font family Helvetica Arial sans serif auto white space nowrap height 42px margin 12px 5px 12px padding 22px text decoration none text align center cursor pointer border border radius 3px vertical align top button span display inline font family Helvetica Arial sans serif text decoration none font weight 500 font style normal font size 16px line height 42px cursor pointer border none border radius 6px poweredWrapper padding 20px margin auto display block span display inline block height 32px line height 32px padding 5px margin 5px 5px clear clear both profile list d

In [14]:
import math
import nltk
from textblob import TextBlob as tb

nltk.download('punkt')

def tf(word, blob):
    return blob.words.count(word) / len(blob.words)

def n_containing(word, bloblist):
    return sum(1 for blob in bloblist if word in blob)

def idf(word, bloblist):
    return math.log(len(bloblist) / (1 + n_containing(word, bloblist)))

def tfidf(word, blob, bloblist):
    return tf(word, blob) * idf(word, bloblist)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\nems\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [15]:
bloblist = []
for i in range(len(example_doc)):
    blob = tb(example_doc[i])
    bloblist.append(blob)
print (len(bloblist))

254


In [16]:
kw_n = 10

key_list = []
for i, blob in enumerate(bloblist):
    scores = {word: tfidf(word, blob, bloblist) for word in blob.words}
    sorted_words = sorted(scores.items(), key=lambda x: x[1], reverse=True)
    tmp = ""
    for word, score in sorted_words[ :kw_n]:
        tmp += word + ' '
    key_list.append(tmp)

In [17]:
import gensim, logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

C:\Users\nems\Anaconda3\lib\site-packages\gensim\utils.py:862: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [18]:
# train word2vec on the two sentences

#model = gensim.models.Word2Vec(example_doc, min_count=1, size=200, hs=1, negative=0)
model = gensim.models.Word2Vec.load("word2vec.sav")

2017-10-29 09:29:17,230 : INFO : loading Word2Vec object from word2vec.sav
2017-10-29 09:29:17,235 : INFO : loading wv recursively from word2vec.sav.wv.* with mmap=None
2017-10-29 09:29:17,237 : INFO : setting ignored attribute syn0norm to None
2017-10-29 09:29:17,239 : INFO : setting ignored attribute cum_table to None
2017-10-29 09:29:17,241 : INFO : loaded word2vec.sav


In [19]:
letter_score = []
for letter in key_list:
    if len(letter) == 0:
        pass
    else:
        score = model.score(letter.split())
        letter_score.append(score.reshape(1, score.shape[0]))

2017-10-29 09:29:26,801 : INFO : scoring sentences with 3 workers on 86 vocabulary and 200 features, using sg=0 hs=1 sample=0.001 and negative=0
2017-10-29 09:29:26,806 : INFO : reached end of input; waiting to finish 1 outstanding jobs
2017-10-29 09:29:26,808 : INFO : scoring 10 sentences took 0.0s, 1892 sentences/s
2017-10-29 09:29:26,811 : INFO : scoring sentences with 3 workers on 86 vocabulary and 200 features, using sg=0 hs=1 sample=0.001 and negative=0
2017-10-29 09:29:26,815 : INFO : reached end of input; waiting to finish 1 outstanding jobs
2017-10-29 09:29:26,817 : INFO : scoring 10 sentences took 0.0s, 2256 sentences/s
2017-10-29 09:29:26,819 : INFO : scoring sentences with 3 workers on 86 vocabulary and 200 features, using sg=0 hs=1 sample=0.001 and negative=0
2017-10-29 09:29:26,822 : INFO : reached end of input; waiting to finish 1 outstanding jobs
2017-10-29 09:29:26,824 : INFO : scoring 10 sentences took 0.0s, 3156 sentences/s
2017-10-29 09:29:26,826 : INFO : scoring se

2017-10-29 09:29:26,991 : INFO : scoring 10 sentences took 0.0s, 1114 sentences/s
2017-10-29 09:29:26,995 : INFO : scoring sentences with 3 workers on 86 vocabulary and 200 features, using sg=0 hs=1 sample=0.001 and negative=0
2017-10-29 09:29:26,999 : INFO : reached end of input; waiting to finish 1 outstanding jobs
2017-10-29 09:29:27,001 : INFO : scoring 10 sentences took 0.0s, 3227 sentences/s
2017-10-29 09:29:27,002 : INFO : scoring sentences with 3 workers on 86 vocabulary and 200 features, using sg=0 hs=1 sample=0.001 and negative=0
2017-10-29 09:29:27,006 : INFO : reached end of input; waiting to finish 1 outstanding jobs
2017-10-29 09:29:27,007 : INFO : scoring 10 sentences took 0.0s, 3551 sentences/s
2017-10-29 09:29:27,009 : INFO : scoring sentences with 3 workers on 86 vocabulary and 200 features, using sg=0 hs=1 sample=0.001 and negative=0
2017-10-29 09:29:27,012 : INFO : reached end of input; waiting to finish 1 outstanding jobs
2017-10-29 09:29:27,013 : INFO : scoring 10

2017-10-29 09:29:27,226 : INFO : reached end of input; waiting to finish 1 outstanding jobs
2017-10-29 09:29:27,231 : INFO : scoring 10 sentences took 0.0s, 929 sentences/s
2017-10-29 09:29:27,234 : INFO : scoring sentences with 3 workers on 86 vocabulary and 200 features, using sg=0 hs=1 sample=0.001 and negative=0
2017-10-29 09:29:27,237 : INFO : reached end of input; waiting to finish 1 outstanding jobs
2017-10-29 09:29:27,239 : INFO : scoring 10 sentences took 0.0s, 3101 sentences/s
2017-10-29 09:29:27,241 : INFO : scoring sentences with 3 workers on 86 vocabulary and 200 features, using sg=0 hs=1 sample=0.001 and negative=0
2017-10-29 09:29:27,244 : INFO : reached end of input; waiting to finish 1 outstanding jobs
2017-10-29 09:29:27,247 : INFO : scoring 10 sentences took 0.0s, 2528 sentences/s
2017-10-29 09:29:27,248 : INFO : scoring sentences with 3 workers on 86 vocabulary and 200 features, using sg=0 hs=1 sample=0.001 and negative=0
2017-10-29 09:29:27,252 : INFO : reached end

2017-10-29 09:29:27,443 : INFO : reached end of input; waiting to finish 1 outstanding jobs
2017-10-29 09:29:27,445 : INFO : scoring 10 sentences took 0.0s, 2724 sentences/s
2017-10-29 09:29:27,448 : INFO : scoring sentences with 3 workers on 86 vocabulary and 200 features, using sg=0 hs=1 sample=0.001 and negative=0
2017-10-29 09:29:27,450 : INFO : reached end of input; waiting to finish 1 outstanding jobs
2017-10-29 09:29:27,452 : INFO : scoring 10 sentences took 0.0s, 3086 sentences/s
2017-10-29 09:29:27,454 : INFO : scoring sentences with 3 workers on 86 vocabulary and 200 features, using sg=0 hs=1 sample=0.001 and negative=0
2017-10-29 09:29:27,461 : INFO : reached end of input; waiting to finish 1 outstanding jobs
2017-10-29 09:29:27,464 : INFO : scoring 10 sentences took 0.0s, 1202 sentences/s
2017-10-29 09:29:27,466 : INFO : scoring sentences with 3 workers on 86 vocabulary and 200 features, using sg=0 hs=1 sample=0.001 and negative=0
2017-10-29 09:29:27,469 : INFO : reached en

2017-10-29 09:29:27,679 : INFO : reached end of input; waiting to finish 1 outstanding jobs
2017-10-29 09:29:27,680 : INFO : scoring 10 sentences took 0.0s, 3187 sentences/s
2017-10-29 09:29:27,682 : INFO : scoring sentences with 3 workers on 86 vocabulary and 200 features, using sg=0 hs=1 sample=0.001 and negative=0
2017-10-29 09:29:27,686 : INFO : reached end of input; waiting to finish 1 outstanding jobs
2017-10-29 09:29:27,687 : INFO : scoring 10 sentences took 0.0s, 3096 sentences/s
2017-10-29 09:29:27,689 : INFO : scoring sentences with 3 workers on 86 vocabulary and 200 features, using sg=0 hs=1 sample=0.001 and negative=0
2017-10-29 09:29:27,693 : INFO : reached end of input; waiting to finish 1 outstanding jobs
2017-10-29 09:29:27,694 : INFO : scoring 10 sentences took 0.0s, 3027 sentences/s
2017-10-29 09:29:27,697 : INFO : scoring sentences with 3 workers on 86 vocabulary and 200 features, using sg=0 hs=1 sample=0.001 and negative=0
2017-10-29 09:29:27,705 : INFO : reached en

2017-10-29 09:29:27,905 : INFO : reached end of input; waiting to finish 1 outstanding jobs
2017-10-29 09:29:27,908 : INFO : scoring 10 sentences took 0.0s, 2380 sentences/s
2017-10-29 09:29:27,911 : INFO : scoring sentences with 3 workers on 86 vocabulary and 200 features, using sg=0 hs=1 sample=0.001 and negative=0
2017-10-29 09:29:27,919 : INFO : reached end of input; waiting to finish 1 outstanding jobs
2017-10-29 09:29:27,925 : INFO : scoring 10 sentences took 0.0s, 847 sentences/s
2017-10-29 09:29:27,928 : INFO : scoring sentences with 3 workers on 86 vocabulary and 200 features, using sg=0 hs=1 sample=0.001 and negative=0
2017-10-29 09:29:27,932 : INFO : reached end of input; waiting to finish 1 outstanding jobs
2017-10-29 09:29:27,933 : INFO : scoring 10 sentences took 0.0s, 3345 sentences/s
2017-10-29 09:29:27,935 : INFO : scoring sentences with 3 workers on 86 vocabulary and 200 features, using sg=0 hs=1 sample=0.001 and negative=0
2017-10-29 09:29:27,939 : INFO : reached end

2017-10-29 09:29:28,168 : INFO : scoring sentences with 3 workers on 86 vocabulary and 200 features, using sg=0 hs=1 sample=0.001 and negative=0
2017-10-29 09:29:28,174 : INFO : reached end of input; waiting to finish 1 outstanding jobs
2017-10-29 09:29:28,176 : INFO : scoring 10 sentences took 0.0s, 3744 sentences/s
2017-10-29 09:29:28,178 : INFO : scoring sentences with 3 workers on 86 vocabulary and 200 features, using sg=0 hs=1 sample=0.001 and negative=0
2017-10-29 09:29:28,184 : INFO : reached end of input; waiting to finish 1 outstanding jobs
2017-10-29 09:29:28,188 : INFO : scoring 10 sentences took 0.0s, 1372 sentences/s
2017-10-29 09:29:28,190 : INFO : scoring sentences with 3 workers on 86 vocabulary and 200 features, using sg=0 hs=1 sample=0.001 and negative=0
2017-10-29 09:29:28,194 : INFO : reached end of input; waiting to finish 1 outstanding jobs
2017-10-29 09:29:28,196 : INFO : scoring 10 sentences took 0.0s, 2308 sentences/s
2017-10-29 09:29:28,198 : INFO : scoring se

In [20]:
fraud_kw_vec = [-0.59686035,-0.6725697,-0.69134709,-0.62265558,-0.59475399,-0.69835925,
                -0.67224162,-0.67393852,-0.67712811,-0.58945206]

In [21]:
# Cosine Similarity

from scipy import spatial

ratio_fraud = 1 - spatial.distance.cosine(fraud_kw_vec[:len(letter_score[-1][0])], letter_score[-1])
print (ratio_fraud)

0.953370926155
